In [2]:
from operator import mod
import os 
import sys


sys.path.append('/home/peterjaq/Project/optical_film_toolbox')

import pandas as pd 
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt 
import plotly
import plotly.graph_objects as go


from openTSNE import TSNE
from openTSNE.callbacks import ErrorLogger
from common.utils.tsne_utils import plot, MACOSKO_COLORS

from models.fcn_ae import FullyConnectedAutoEncoder
from refractivesqlite import dboperations as DB

In [4]:
save_model = True
show_result = True
show_tSNE = True 

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


In [6]:
rangeMin = 0.3
rangeMax = 1.0
re = 10

dbpath = "../db_sqlite/refractive.db"
db = DB.Database(dbpath)
matList = db.search_custom('select * from pages')
matPageId = list()
for mat in matList:
  if mat[-3] < rangeMin and mat[-2] > rangeMax:
    print(mat)
    matPageId.append(mat[0])
    
print(matPageId)

'main', 'Pb', 'Werner', 'main/Pb/Werner.yml', 1, 1, 0.017586, 2.479684, 150)
(500, 'main', 'Pb', 'Werner-DFT', 'main/Pb/Werner-DFT.yml', 1, 1, 0.017586, 2.479684, 150)
(501, 'main', 'Pd', 'Johnson', 'main/Pd/Johnson.yml', 1, 1, 0.188, 1.937, 49)
(502, 'main', 'Pd', 'Palm', 'main/Pd/Palm.yml', 1, 1, 0.250019531, 1.684528687, 670)
(503, 'main', 'Pd', 'Werner', 'main/Pd/Werner.yml', 1, 1, 0.017586, 2.479684, 150)
(505, 'main', 'Pd', 'Rakic-BB', 'main/Pd/Rakic-BB.yml', 1, 1, 0.24797, 12.398, 200)
(506, 'main', 'Pd', 'Rakic-LD', 'main/Pd/Rakic-LD.yml', 1, 1, 0.24797, 12.398, 1000)
(507, 'main', 'Pd', 'Werner-DFT', 'main/Pd/Werner-DFT.yml', 1, 1, 0.017586, 2.479684, 150)
(510, 'main', 'Pt', 'Werner', 'main/Pt/Werner.yml', 1, 1, 0.017586, 2.479684, 150)
(511, 'main', 'Pt', 'Rakic-BB', 'main/Pt/Rakic-BB.yml', 1, 1, 0.24797, 12.398, 200)
(512, 'main', 'Pt', 'Rakic-LD', 'main/Pt/Rakic-LD.yml', 1, 1, 0.24797, 12.398, 1000)
(513, 'main', 'Pt', 'Werner-DFT', 'main/Pt/Werner-DFT.yml', 1, 1, 0.017586

In [8]:
input_data = {}
input_id = {}
for matId in matPageId:
  mat = db.get_material(matId)
  material_name = mat.pageinfo['book'] + '_' + mat.pageinfo['page']
  material_n = []
  material_k = []

  for i in range(int(rangeMin*1000), int(rangeMax*1000), re):
    try:
        material_n.append(mat.get_refractiveindex(i))
    except:
        material_n.append(0)
    try:
      material_k.append(mat.get_extinctioncoefficient(i))
    except:
      material_k.append(0)

  input_data[material_name] = np.concatenate((np.array(material_n), np.array(material_k)))
  input_id[material_name] = matId

num_epochs = 600
batch_size = 256
batch_losses = []


train_dataset = tf.data.Dataset.from_tensor_slices(list(input_data.values())).shuffle(1).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices({'name': list(input_data.keys()),
                                                   'value':list(input_data.values())}).batch(1)

Material main/Ag/Johnson.yml loaded.
Material main/Ag/Yang.yml loaded.
Material main/Ag/Babar.yml loaded.
Material main/Ag/Werner.yml loaded.
Material main/Ag/Hagemann.yml loaded.
Material main/Ag/Ciesielski.yml loaded.
Material main/Ag/Ciesielski-Ge.yml loaded.
Material main/Ag/Ciesielski-Ni.yml loaded.
Material main/Ag/Rakic-BB.yml loaded.
Material main/Ag/Rakic-LD.yml loaded.
Material main/Ag/Werner-DFT.yml loaded.
Material main/Al/Rakic.yml loaded.
Material main/Al/McPeak.yml loaded.
Material main/Al/Hagemann.yml loaded.
Material main/Al/Rakic-BB.yml loaded.
Material main/Al/Rakic-LD.yml loaded.
Material main/AlAs/Rakic.yml loaded.
Material main/GaAs/Adachi.yml loaded.
Material main/GaAs/Rakic.yml loaded.
Material main/GaAs/Ozaki.yml loaded.
Material main/InAs/Adachi.yml loaded.
Material main/Au/Johnson.yml loaded.
Material main/Au/Babar.yml loaded.
Material main/Au/Werner.yml loaded.
Material main/Au/Hagemann.yml loaded.
Material main/Au/Ciesielski.yml loaded.
Material main/Au/Cie

In [10]:
model = FullyConnectedAutoEncoder(latent_dim=20, input_shape=int(((1000-300)/10)*2))

optimizer = tf.keras.optimizers.Adam(1e-4)
global_step = tf.Variable(0)

for epoch in range(num_epochs):
    for step,x in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            x_rec_logits = model(x)
            rec_loss =tf.losses.mean_squared_error(x,x_rec_logits)
            batch_losses.append(rec_loss)
            rec_loss =tf.reduce_mean(rec_loss)
        
        grads=tape.gradient(rec_loss,model.trainable_variables)
        optimizer.apply_gradients(zip(grads,model.trainable_variables))
        if step%100==0:
            print(epoch, step, float(rec_loss))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

0 0 7.179900169372559
1 0 6.7280659675598145
2 0 6.350732803344727
3 0 5.974738121032715
4 0 5.628915786743164
5 0 5.315147876739502
6 0 5.001900672912598
7 0 4.672431945800781
8 0 4.3376874923706055
9 0 4.012472152709961
10 0 3.699296236038208
11 0 3.39315128326416
12 0 3.093524932861328
13 0 2.8060154914855957
14 0 2.535547971725464
15 0 2.282602310180664
16 0 2.046499729156494
17 0 1.827989101409912
18 0 1.6293450593948364
19 0 1.452449083328247
20 0 1.2963056564331055
21 0 1.1583867073059082
22 0 1.

In [12]:
if show_tSNE:
  tSNE_result = []

  tsne = TSNE(
    perplexity=10,
    metric="euclidean",
    n_jobs=4,
    random_state=42,
    negative_gradient_method="fft",
  )


  for test in test_dataset:
    # print(test)
    name = test['name']
    result = list(model.encode(test['value']))[0]
    tSNE_result.append(result)

  embedding_train = tsne.fit(np.array(tSNE_result))
  # print(embedding_train)
  # plot(embedding_train, list(input_data.keys()))
  # plt.savefig('/home/jizhidemifan/Project/optical_film_toolbox/figure/tSNE.png')

embedding_y = [v[1] for v in embedding_train]
embedding_color = [abs(v[0]) + abs(v[1]) for v in embedding_train]


In [14]:
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import normalize


ss = QuantileTransformer()
embedding_train_sclar= ss.fit_transform(embedding_train)

embedding_x = [v[0] for v in embedding_train_sclar]
embedding_y = [v[1] for v in embedding_train_sclar]

# embedding_color = [abs(v[0]) + abs(v[1]) for v in embedding_train_sclar]

fig = go.Figure(data=go.Scatter(
                                x=embedding_x,
                                y=embedding_y,
                                text=list(input_data.keys()),
                                marker_color=embedding_color,
                                marker_colorscale='Viridis',
                                marker_size=8,
                                mode='markers'
))

fig.update_layout(
    template="plotly_white",
    autosize=False,
    width=500,
    height=500)

fig.show()
fig.write_html('../figure/tsne/20-hidden-debug.html')

In [22]:
with open('./encode_data.txt', 'w') as f:
    for id, name, x, y in zip(list(input_id.values()), list(input_data.keys()), embedding_x, embedding_y):
        f.write(f'{id} {name} {x} {y}\n')
        print(f'{id} {name} {x} {y}')



79792746114 0.31865284974093266
106 Bi_Werner-DFT 0.8212435233160622 0.6683937823834196
108 CsBr_Li 0.30310880829015546 0.6968911917098446
109 CsBr_Querry 0.10621761658031088 0.45077720207253885
111 KBr_Li 0.03367875647668394 0.5051813471502591
112 LiBr_Li 0.2927461139896373 0.6476683937823833
113 NaBr_Li 0.04922279792746114 0.533678756476684
114 RbBr_Li 0.031088082901554404 0.5025906735751295
117 C_Phillip 0.5155440414507773 0.7564766839378239
120 C_Hagemann 0.5 0.5414507772020726
121 C_Larruquert 0.5259067357512953 0.7642487046632125
123 C_Djurisic-o 0.5414507772020726 0.4844559585492228
125 C_Querry-DixonKS2 0.48704663212435234 0.5103626943005182
126 C_Querry-DixonHPN2 0.5051813471502591 0.5207253886010363
127 C_Querry-Dixon200-10 0.5103626943005182 0.5181347150259054
128 C_Querry-AsburyMicro260 0.49481865284974097 0.5155440414507773
129 C_Querry-Dixon1102 0.49740932642487046 0.5259067357512953
130 C_Querry-Asbury3222 0.5077720207253886 0.538860103626943
131 C_Querry-Pyrolytic 0.481